In [1]:
# !pip install argparse

In [14]:
import argparse
import os
import re
import time
from openai import OpenAI
from dotenv import load_dotenv
import tiktoken
import requests
import json
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor

In [15]:
# import sys
# import argparse

# # Backup the real `sys.argv`
# original_argv = sys.argv.copy()

# # Set the sys.argv to mimic command line input
# sys.argv = ['script_name', '--context', 'Example context', '--train_output_filename', 'train_output.txt', 
#             '--test_output_filename', 'test_output.txt', '--api', 'openai', '--temperature', '0.7', 
#             '--chunks_to_process', 'all']

# parser = argparse.ArgumentParser(description='Generate QA pairs using specified API, temperature, and process option.')
# parser.add_argument('--context', type=str, required=True, help='One sentence describing the context for the dataset.')
# parser.add_argument('--train_output_filename', type=str, required=True)
# parser.add_argument('--test_output_filename', type=str, required=True)
# parser.add_argument('--api', type=str, choices=['openai', 'runpod'], required=True, help='API to use (openai or runpod)')
# parser.add_argument('--temperature', type=float, default=0.1, help='Temperature for generation (between 0 and 1.2)')
# parser.add_argument('--chunks_to_process', type=str, choices=['one', 'all'], required=True, help='Process one chunk or all chunks')
# args = parser.parse_args()

# # Restore the original `sys.argv`
# sys.argv = original_argv


In [16]:
# Use arguments
train_output_filename = 'train_output.txt'
test_output_filename = 'test_output.txt'
api_choice = 'openai'
temperature = 0.01
chunks_to_process = 'all'
context = 'Spirit Airlines Flight Attendant Company Regulations and Rules.'


In [17]:
print(train_output_filename)
print(test_output_filename)
print(api_choice)
print(temperature)
print(chunks_to_process)
print(context)

train_output.txt
test_output.txt
openai
0.01
all
Spirit Airlines Flight Attendant Company Regulations and Rules.


In [18]:
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

In [19]:
# context
tokens_per_question = 45 # reduce this parameter to increase the granularity of questions. If you reduce this too much the language model may hallucinate content.
# chunk_size = min(model_context_length / (1 + 60/tokens_per_question),25*tokens_per_question)/2 # there are empirically about 60 tokens per QA pair. Also, GPT gets confused making too many questions up.
chunk_size = 1024 # for testing, set a smaller chunk size.
model_context_length = 2048 #This needs to be at least twice the chunk size, otherwise responses will be truncated.

train_sample = "What is the meaning of NOI in the context of Flight Attendants?\nNOI stands for Notice of Investigation, which is a formal document issued by the Company to a Flight Attendant when an incident or violation has occurred.\n\nWhat happens if a Flight Attendant does not pass the second scheduled Recurrent Training class?\nIf a Flight Attendant does not pass the second scheduled Recurrent Training class, they will be placed on inactive, unpaid status, scheduled for an NOI and subject to termination.\n\nWhat is the difference between Line Pay and Block Pay under the Spirit CBA?\nLine Pay is calculated based on the actual time spent flying, while Block Pay is calculated based on the scheduled flight time.\n\nWhat is the purpose of Scheduled and Actual On Duty Limitations in the Spirit CBA?\nSection 7.B.1 Scheduled On Duty Limitations outlines that a Flight Attendant will not be scheduled or re-scheduled to remain on duty for more than fourteen (14) hours, with certain exceptions mentioned in the text. Section 7.B.2 Actual On Duty Limitations outlines that a Flight attendant may remain on duty up to 16 hours due to delays or other events. These can be waived by asking to extend or being asked to extend under a case by case basis."
test_sample = "What is the meaning of NOI in the context of Flight Attendants?\nNOI stands for Notice of Investigation, which is a formal document issued by the Company to a Flight Attendant when an incident or violation has occurred.\n\nWhat happens if a Flight Attendant does not pass the second scheduled Recurrent Training class?\nIf a Flight Attendant does not pass the second scheduled Recurrent Training class, they will be placed on inactive, unpaid status, scheduled for an NOI and subject to termination.\n\nWhat is the difference between Line Pay and Block Pay under the Spirit CBA?\nLine Pay is calculated based on the actual time spent flying, while Block Pay is calculated based on the scheduled flight time.\n\nWhat is the purpose of Scheduled and Actual On Duty Limitations in the Spirit CBA?\nSection 7.B.1 Scheduled On Duty Limitations outlines that a Flight Attendant will not be scheduled or re-scheduled to remain on duty for more than fourteen (14) hours, with certain exceptions mentioned in the text. Section 7.B.2 Actual On Duty Limitations outlines that a Flight attendant may remain on duty up to 16 hours due to delays or other events. These can be waived by asking to extend or being asked to extend under a case by case basis."

questions_per_chunk_train = int(chunk_size / tokens_per_question)
questions_per_chunk_test = max(int(questions_per_chunk_train / 10),1)

print(f'Setting {questions_per_chunk_train} questions per {int(chunk_size)}-token chunk for QA train dataset generation.')


Setting 22 questions per 1024-token chunk for QA train dataset generation.


What is the purpose of Scheduled and Actual On Duty Limitations in the Spirit CBA?\nSection 7.B.1 Scheduled On Duty Limitations outlines that a Flight Attendant will not be scheduled or re-scheduled to remain on duty for more than fourteen (14) hours, with certain exceptions mentioned in the text. Section 7.B.2 Actual On Duty Limitations outlines that a Flight attendant may remain on duty up to 16 hours due to delays or other events. These can be waived by asking to extend or being asked to extend under a case by case basis.


In [20]:

def count_tokens(text):
    encoding = tiktoken.get_encoding("cl100k_base")
    token_count = sum(1 for _ in encoding.encode(text))
    return token_count

def read_and_chunk_txt(file_path):
    chunks = []
    chunk = ""
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            text = line.strip()
            if count_tokens(chunk + text) > chunk_size:
                chunks.append(chunk.strip())
                chunk = text
            else:
                chunk += " " + text
    if chunk:
        chunks.append(chunk.strip())
    return chunks

In [21]:
chunks = read_and_chunk_txt("data/raw_train.txt")

total_tokens = sum(count_tokens(chunk) for chunk in chunks)
print(f"Total tokens in all chunks: {total_tokens}")

estimated_input_tokens = total_tokens * 1.1
estimated_output_tokens = total_tokens * 50/tokens_per_question
total_estimated_tokens = estimated_input_tokens + estimated_output_tokens

estimated_cost_gpt4 = (estimated_input_tokens / 1000 * 0.03) + (estimated_output_tokens  / 1000 * 0.06)
estimated_cost_gpt35turbo = (estimated_input_tokens / 1000 * 0.0005) + (estimated_output_tokens  / 1000 * 0.0015)
print(f"Estimated cost with gpt-4: ${estimated_cost_gpt4:.2f}")
print(f"Estimated cost with gpt-3.5-turbo-16k: ${estimated_cost_gpt35turbo:.2f}")

Total tokens in all chunks: 58885
Estimated cost with gpt-4: $5.87
Estimated cost with gpt-3.5-turbo-16k: $0.13


In [22]:

# while True:
#     if chunks_to_process in ['one', 'all']:
#         break
#     else:
#         print("Invalid option. Please enter 'one' or 'all'.")

# snippets = [
#     f"Provide {questions_per_chunk_train} question and answer pair(s) based on the text above. The question should include sufficient information for the answer, without the user having any further context. The answers need not necessarily borrow verbatim from the input text, but they should maintain the meaning. Vary the style and format of questions. Include some tricky and nuanced questions. In certain answers, reverse the order of words compared to how they appear in the input text. Respond in plain text on a new line for each question and answer. Do not include question numbers. Here is an example of a question answer pair:\n\n<example>\n\n{train_sample}\n\n</example>\n\n",
#     f"Provide {questions_per_chunk_test} question and answer pair(s) based on the text above. The question should include sufficient information for the answer, without the user having any further context. The answers should NOT borrow verbatim from the text above, but they should maintain the meaning. Vary the style and format of questions. Respond in plain text on a new line for each question and answer. Do not include question numbers. Here is an example of a question answer pair:\n\n<example>\n\n{test_sample}\n\n</example>\n\n"
# ]

while True:
    if chunks_to_process in ['one', 'all']:
        break
    else:
        print("Invalid option. Please enter 'one' or 'all'.")

snippets = [
    f"Provide ample question and answer pair(s) based on the text above to fully cover each aspect. The question should include sufficient information for the answer, without the user having any further context. The answers need not necessarily borrow verbatim from the input text, but they should maintain the meaning. Vary the style and format of questions, utilizing first person and third person. Include some less obvious and nuanced questions. In certain answers, reverse the order of words compared to how they appear in the input text. Respond in plain text on a new line for each question and answer. Do NOT include question numbers. Here is an example of a question answer pair:\n\n<example>\n\n{train_sample}\n\n</example>\n\n",
    f"Provide ample question and answer pair(s) based on the text above to fully cover each aspect. The question should include sufficient information for the answer, without the user having any further context. The answers should NOT borrow verbatim from the text above, but they should maintain the meaning. Vary the style and format of questions. Respond in plain text on a new line for each question and answer. Do not include question numbers. Here is an example of a question answer pair:\n\n<example>\n\n{test_sample}\n\n</example>\n\n"
]


In [23]:
# # Function to validate the response format
# def is_valid_qa_format(text):
#     # Split the text into lines
#     lines = text.strip().split('\n')
    
#     # Check for at least one question and one answer
#     has_question = any(line.endswith('?') for line in lines)
#     # has_answer = len(lines) > 1 and any(not line.endswith('?') for line in lines)  # Assuming answer lines don't end with a question mark
    
#     # Basic syntax safety check (example: avoid null bytes which can be problematic)
#     syntax_safe = all('\0' not in line for line in lines)
    
#     return has_question and syntax_safe

# def clean_text(text):
#     # Remove control characters except newline
#     text = re.sub(r'[\x00-\x09\x0b-\x1F\x7F]', '', text)
    
#     # Escape potentially dangerous characters or sequences
#     # This is an example; adapt based on your context
#     text = re.sub(r'[<>{};`]', '', text)
    
#     return text

# # Function to log errors
# def log_error(prompt, response, filename="error_log.txt"):
#     with open(filename, "a", encoding='utf-8') as error_file:
#         error_file.write(f"Prompt: {prompt}\nResponse: {response}\n\n")


In [24]:
# Function to validate the response format
def is_valid_qa_format(text):
    # Split the text into lines
    lines = text.strip().split('\n')
    
    # Check for at least one question and one answer
    has_question = any(line.endswith('?') for line in lines)
    # has_answer = len(lines) > 1 and any(not line.endswith('?') for line in lines)  # Assuming answer lines don't end with a question mark
    
    # Basic syntax safety check (example: avoid null bytes which can be problematic)
    syntax_safe = all('\0' not in line for line in lines)
    
    return has_question and syntax_safe

def clean_text(text):
    # Remove control characters except newline
    text = re.sub(r'[\x00-\x09\x0b-\x1F\x7F]', '', text)
    
    # Escape potentially dangerous characters or sequences
    # This is an example; adapt based on your context
    text = re.sub(r'[<>{};`]', '', text)
    
    return text

# Function to log errors
def log_error(prompt, response, filename="error_log.txt"):
    with open(filename, "a", encoding='utf-8') as error_file:
        error_file.write(f"Prompt: {prompt}\nResponse: {response}\n\n")

# Update your loop for processing chunks
for idx, snippet in enumerate(snippets):
    # print(snippet)
    output_filename = f"data/{train_output_filename}" if idx == 0 else f"data/{test_output_filename}"
    
    with open(output_filename, "a", encoding='utf-8') as output_file:
        if api_choice == "openai":
            for chunk_idx, chunk in enumerate(chunks):
                prompt = f"<input-text>\n\nContext: {context}\n\nText:\n\n{chunk}\n\n</input-text>\n\n{snippet}"
                if chunks_to_process == 'one' and chunk_idx > 0:
                    break
                if chunks_to_process == 'one':
                    print(f"The prompt is:\n\n{prompt}")

                completion = client.chat.completions.create(
                    model="TheBloke/CapybaraHermes-2.5-Mistral-7B-GGUF",
                    temperature=temperature,
                    messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ],
                )
                max_tokens=int(model_context_length * 0.9) # Ensure you have defined max_tokens appropriately)

                response = completion.choices[0].message.content
                # print(f"\n\nRaw Response:\n\n{response}")
                
                cleaned_response = clean_text(response)

                # print(f"\n\nCleaned Response:\n\n{cleaned_response}")

                if is_valid_qa_format(cleaned_response):
                    output_file.write(cleaned_response + "\n\n")
                else:
                    log_error(prompt, cleaned_response, filename=f"error_log_{idx}.txt") # Log the error with a unique file per snippet

                output_file.flush()
                time.sleep(0.2)

        elif api_choice == "runpod":
            max_tokens = int(model_context_length * 0.9)
            if chunks_to_process == 'all':
                with ThreadPoolExecutor(max_workers=8) as executor:
                    prompt = f"<input-text>\n\nContext: {context}\n\nText:\n\n{chunk}\n\n</input-text>\n\n{snippet}"
                    future_to_chunk = {executor.submit(query_runpod, pod_id, prompt, max_tokens, temperature): chunk for chunk in chunks}
                    for future in concurrent.futures.as_completed(future_to_chunk):
                        chunk = future_to_chunk[future]
                        try:
                            response = future.result()
                        except Exception as exc:
                            print(f"Generated an exception: {exc}")
                        else:
                            if is_valid_qa_format(response):
                                output_file.write(response + "\n\n")
                            else:
                                log_error(chunk, response, filename=f"error_log_{idx}.txt") # Log the error with a unique file per snippet

                            output_file.flush()
            else:
                for chunk in chunks:
                    prompt = f"<input-text>\n\nContext: {context}\n\nText:\n\n{chunk}\n\n</input-text>\n\n{snippet}"
                    # print(f"Prompt:\n\n{prompt}")
                    response = query_runpod(pod_id, prompt, max_tokens, temperature)
                    # print(f"\n\nResponse:\n\n{response}")
                    cleaned_response = clean_text(response)
                    # print(f"Cleaned Response:\n\n{cleaned_response}")
                    if is_valid_qa_format(cleaned_response):
                        output_file.write(cleaned_response + "\n\n")
                    else:
                        log_error(chunk, cleaned_response, filename=f"error_log_{idx}.txt")

                    output_file.flush()

                    if chunks_to_process == 'one':
                        break

The above needs to be run in .venv to work

EDIT ME TO COPY WITH FORMATTING

<!-- Run 1:
api_choice = 'openai'
temperature = 0.01
chunks_to_process = 'all'
context = 'Spirit Airlines Flight Attendant Collective Bargaining Agreement and Rules.'

# context
tokens_per_question = 45 # reduce this parameter to increase the granularity of questions. If you reduce this too much the language model may hallucinate content.
# chunk_size = min(model_context_length / (1 + 60/tokens_per_question),25*tokens_per_question)/2 # there are empirically about 60 tokens per QA pair. Also, GPT gets confused making too many questions up.
chunk_size = 2000 # for testing, set a smaller chunk size.
model_context_length = 4000 #This needs to be at least twice the chunk size, otherwise responses will be truncated. -->

<!-- Run 2:
api_choice = 'openai'
temperature = 0.05
chunks_to_process = 'all'
context = 'Spirit Airlines Flight Attendant Collective Bargaining Agreement and Rules.'

# context
tokens_per_question = 45 # reduce this parameter to increase the granularity of questions. If you reduce this too much the language model may hallucinate content.
# chunk_size = min(model_context_length / (1 + 60/tokens_per_question),25*tokens_per_question)/2 # there are empirically about 60 tokens per QA pair. Also, GPT gets confused making too many questions up.
chunk_size = 2000 # for testing, set a smaller chunk size.
model_context_length = 4000 #This needs to be at least twice the chunk size, otherwise responses will be truncated. -->

<!-- Run 3:
api_choice = 'openai'
temperature = 0.1
chunks_to_process = 'all'
context = 'Spirit Airlines Flight Attendant Contractual Regulations and Rules.'

# context
tokens_per_question = 45 # reduce this parameter to increase the granularity of questions. If you reduce this too much the language model may hallucinate content.
# chunk_size = min(model_context_length / (1 + 60/tokens_per_question),25*tokens_per_question)/2 # there are empirically about 60 tokens per QA pair. Also, GPT gets confused making too many questions up.
chunk_size = 2000 # for testing, set a smaller chunk size.
model_context_length = 4000 #This needs to be at least twice the chunk size, otherwise responses will be truncated. -->

<!-- Run 4:
api_choice = 'openai'
temperature = 0.15
chunks_to_process = 'all'
context = 'Spirit Airlines Flight Attendant Contractual Regulations and Flight Attendant Rules.'

# context
tokens_per_question = 55 # reduce this parameter to increase the granularity of questions. If you reduce this too much the language model may hallucinate content.
# chunk_size = min(model_context_length / (1 + 60/tokens_per_question),25*tokens_per_question)/2 # there are empirically about 60 tokens per QA pair. Also, GPT gets confused making too many questions up.
chunk_size = 2000 # for testing, set a smaller chunk size.
model_context_length = 4000 #This needs to be at least twice the chunk size, otherwise responses will be truncated. -->

<!-- Run 5:
api_choice = 'openai'
temperature = 0.2
chunks_to_process = 'all'
context = 'Spirit Airlines Flight Attendant Company Regulations and Flight Attendant Rules.'

# context
tokens_per_question = 75 # reduce this parameter to increase the granularity of questions. If you reduce this too much the language model may hallucinate content.
# chunk_size = min(model_context_length / (1 + 60/tokens_per_question),25*tokens_per_question)/2 # there are empirically about 60 tokens per QA pair. Also, GPT gets confused making too many questions up.
chunk_size = 2000 # for testing, set a smaller chunk size.
model_context_length = 4000 #This needs to be at least twice the chunk size, otherwise responses will be truncated. -->

<!-- Run 6:
api_choice = 'openai'
temperature = 0.25
chunks_to_process = 'all'
context = 'Spirit Airlines Flight Attendant Company Regulations and Rules.'

# context
tokens_per_question = 100 # reduce this parameter to increase the granularity of questions. If you reduce this too much the language model may hallucinate content.
# chunk_size = min(model_context_length / (1 + 60/tokens_per_question),25*tokens_per_question)/2 # there are empirically about 60 tokens per QA pair. Also, GPT gets confused making too many questions up.
chunk_size = 2000 # for testing, set a smaller chunk size.
model_context_length = 4000 #This needs to be at least twice the chunk size, otherwise responses will be truncated. -->

<!-- Run 7:
api_choice = 'openai'
temperature = 0.3
chunks_to_process = 'all'
context = 'Spirit Airlines Flight Attendant Company Regulations and Rules.'

# context
tokens_per_question = 125 # reduce this parameter to increase the granularity of questions. If you reduce this too much the language model may hallucinate content.
# chunk_size = min(model_context_length / (1 + 60/tokens_per_question),25*tokens_per_question)/2 # there are empirically about 60 tokens per QA pair. Also, GPT gets confused making too many questions up.
chunk_size = 2000 # for testing, set a smaller chunk size.
model_context_length = 4000 #This needs to be at least twice the chunk size, otherwise responses will be truncated. -->